### Introduction
This notebook aims to use transfer learing to working as image matching tools for fashion.

### Task

There are two groups of photos, one group(targets) repersent the different products avable in, another group(querires) is actual photos taken by employees and user. The tasks is to pair the queriry photo to corresponds photo in target group.

In the seeting of this task,one querirring seesion may contain few hundredrs querires and few hundredrs targets. But the targets may change over each session.

(see the moviation part belowe for such wired setting for the task)

### Basic Idea

This image pairing probelm can be restated as finding the most similar photo of query in targets set.\
The photo is first embebed into a vector space, then simirity can then be defined as the angle between the two vectors.\
The inititial trial is to use a pretrained featurizer to embed the image, then identify the target photo with max simirity.

(note: more potential varyation exist, see discussion for detail)

In [2]:
# helper code and importssssssss
import torch
from torch import nn
from torchvision import transforms, models

from PIL import Image
from matplotlib import pyplot as plt
from pathlib import Path

#main test code
def main():
    data_path = Path("./UNIQOExample")
    show_architecture = False
    show_result_by_graph = False
    result = []
    transforms_default = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize(224, antialias=True),
    ])
    transforms_maxvit_t = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize(224, antialias=True),
    ])
    models_to_try = [
        (models.alexnet, models.AlexNet_Weights.DEFAULT, transforms_default), 
        (models.densenet121, models.DenseNet121_Weights.DEFAULT, transforms_default),
        (models.convnext_tiny, models.ConvNeXt_Tiny_Weights.DEFAULT, transforms_default), 
        (models.efficientnet_b0, models.EfficientNet_B0_Weights.DEFAULT, transforms_default), 
        (models.googlenet, models.GoogLeNet_Weights.DEFAULT, transforms_default),
        (models.inception_v3, models.Inception_V3_Weights.DEFAULT, transforms_default),
        (models.maxvit_t, models.MaxVit_T_Weights.DEFAULT, transforms_maxvit_t), #used 224 as input size
        (models.mnasnet0_5, models.MNASNet0_5_Weights.DEFAULT, transforms_default),
        (models.mobilenet_v3_small, models.MobileNet_V3_Small_Weights.DEFAULT, transforms_default),
    ] #tuple(model_constructor, pretrain_wieghts, transforms for inputs image)

    for model_constructor, wieghts, transforms_needed in models_to_try:
        
        # make model by architecture specified and turn off the classfier in the orginal model
        model = make_features_extractor(model_constructor, wieghts)
        if show_architecture: 
            print(model)
        
        (inputs, input_files, labels), (match_targets, match_files, match_labels) = load_data(data_path, transforms_needed)
        
        # featurize input and match samples, then normalize to 1(by row aka each sample)
        outputs = featurize(model, inputs)
        match_sapce = featurize(model, match_targets)

        # compute cos_sim for all input with and match target
        # shape of torch.mm = (# of inputs, # of features) * (# of features, # of match samples) 
        # shape of torch.mm = (# of inputs, # of match samples)
        match_sapce = torch.transpose(match_sapce, dim0=0, dim1=1)
        cos_similaritys, top_classes = torch.mm(outputs, match_sapce).topk(1)
        
        # match the predecition to file name of match samples
        top_classes = torch.tensor( list(match_labels[i.item()] for i in top_classes))
        
        accuracy = labels == top_classes
        accuracy = (accuracy.sum()/accuracy.shape[0]).item()
        result.append((model._get_name(), accuracy))

        print(f"{model.__class__.__name__}:\t accuracy = {accuracy*100:.2f} %")
        if show_result_by_graph: 
            show_helper(input_files, labels, top_classes)

class Identity(nn.Module):
    def __init__(self) -> None:
        super().__init__()
    
    def forward(self, x):
        return x

def make_features_extractor(model_constructor, wieghts):
    """constures a model

    Args:
        model_constructor (_type_): _description_
        wieghts (_type_): _description_

    Raises:
        NotImplementedError: _description_
        NotImplementedError: _description_

    Returns:
        _type_: _description_
    """
    
    if wieghts is None:
        model = model_constructor(pretrained = True)
    model = model_constructor(weights = wieghts)
    model.eval()
    
    if hasattr(model, "classifier") and not hasattr(model, "fc"):
        model.classifier = Identity()
    elif not hasattr(model, "classifier") and hasattr(model, "fc"):
        model.fc = Identity()
    elif not hasattr(model, "classifier") and not hasattr(model, "fc"):
        raise NotImplementedError(f"{model.__class__.__name__} have no features layer nor fc layer")
    else:
        raise NotImplementedError(f"ERROR, {model.__class__.__name__} have both????????")
    
    for param in model.parameters():
            param.requires_grad = False

    return model

def load_data(data_path, transforms_needed = None):
    """load data deom datapath and transform

    Args:
        (data_path: Path): path object
        (transforms_needed:torchvision.transforms): transforms the import files

    Returns:
        (inputs: tensor(B, C, H, W), input_files: list[Path], labels: list[int]), (match_targets: tensor(B, C, H, W), match_labels: list[int]): as shown
    """
    
    if transforms_needed is None:
        default_transforms = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize(224, antialias=True),
        ]) 
        transforms_needed = default_transforms
    
    inputs_path = data_path/"queries"
    inputs, labels, input_files = [], [], []
    for file in inputs_path.glob("*/*"):
        input_files.append(file)
        labels.append(int(file.parts[-2]))
        inputs.append(transforms_needed(Image.open(file)))
    inputs = torch.stack(inputs)
    labels = torch.tensor(labels)

    match_target_path = data_path/"targets"
    match_targets, match_files, match_labels= [], [], []
    for file in match_target_path.glob("*"):
        match_files.append(file)
        match_labels.append(int(file.stem))
        match_targets.append(transforms_needed(Image.open(file)))
    match_targets = torch.stack(match_targets)
    match_labels = torch.tensor(match_labels)

    return (inputs, input_files, labels), (match_targets, match_files, match_labels)

def featurize(model: torch.nn.Module, inputs: torch.Tensor) -> torch.Tensor:
    """take inputs tensor and do forward() to featurize the inputs image tensor, 
    as different model have diffent output shape after featurise, this function also adjust the shpae
    
    Args:
        model (torch.nn.Module): model
        inputs (torch.Tensor): (B, C, H, W) C ==3

    Raises:
        NotImplementedError: Unexcepted shape of output

    Returns:
        torch.Tensor: _description_
    """
    outputs = model(inputs)
    if outputs.dim() == 4:
        outputs = torch.nn.functional.adaptive_avg_pool2d(outputs, output_size=(1,1))
        outputs = torch.squeeze(outputs, dim = tuple(range(2, outputs.dim())))
    elif outputs.dim() == 2:
        pass
    else:
        raise NotImplementedError(f"Unexcepted deminsion shape {outputs.shape} from {model.__class__.__name__}")
    outputs = nn.functional.normalize(outputs, p=2, dim=1)
    return outputs

def show_helper(input_files, labels, top_classes):
    """some helper code for visualztion

    Args:
        input_files (_type_): _description_
        labels (_type_): _description_
        top_classes (_type_): _description_
    """
    for file, label, predecition in zip(input_files, labels, top_classes):
        
        label, predecition = label.item(), predecition.item()

        predecition = Image.open(file.parents[2]/"targets"/(str(predecition)+".jpg"))
        file = Image.open(file)
        # label = Image.open(Path(data_path)/"target"/f"{label}.jpg")
        
        fig = plt.figure(figsize=(10,7))
        fig.add_subplot(1, 2, 1)
        plt.imshow(file)
        fig.add_subplot(1, 2, 2)
        plt.imshow(predecition)

#driver
if __name__ == "__main__":
    main()

AlexNet:	 accuracy = 66.67 %
DenseNet:	 accuracy = 73.33 %
ConvNeXt:	 accuracy = 46.67 %
EfficientNet:	 accuracy = 80.00 %
GoogLeNet:	 accuracy = 60.00 %
Inception3:	 accuracy = 66.67 %
MaxVit:	 accuracy = 66.67 %
MNASNet:	 accuracy = 66.67 %
MobileNetV3:	 accuracy = 60.00 %


## Discussion


### Future work on
1. if labeled data is avalable, try SIAMESE CNN
2. Use other kinds of meteric to determin the simirity, i.e. L2-norm, L1-norm for non-unity vectors

## Reference

### Real Motivation
In a road trip with my friends, one of my friends who own a fashion bussess needed to work in hotel room, while watching he works, I found that most of his work are pretty mechanical. There are two major taks:
1. Product matching, match the product photo taken by employee to sample photo from supplier for interal invetory system.
2. Table recognition, manul input sale data from photo

"it should be a easy task for AI" I thougth it should be easy to bulid a tool with existig resourse. \
I was worng. VERY WRONG\
Atleast not easy for me who know nothing abour programming at that time.\

The table recognition task, i tried few online tools and most of them are horrible at idenfity hand-written table. Then I switched to use paddlepaddle(a ML package by China internet giant  BAIDU)